In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import simfin as sf
from simfin.names import *

In [ ]:
# Set the SimFin data-directory
sf.set_data_dir("C:\\dev\\trading\\tradesystem1\\data\\simfin")

In [ ]:
#  Set your API Key
sf_api_key = "<your API key>"
sf.set_api_key(sf_api_key)

In [ ]:
#  Configure SimFin hub with cache refresh params
variant = 'latest' #  latest or ttm
hub = sf.StockHub(market='us',
                  refresh_days=30,
                  refresh_days_shareprices=1)


In [ ]:
#  Load quartely income in line line!!!
income_df = hub.load_income(variant='quarterly')  # annual or quarterly
income_df = income_df.reset_index()
for col in income_df.columns:
    print(col)
income_df['INCOME_GROWTH'] = income_df[OPERATING_INCOME].pct_change(periods=1)
#  Average income growth in the last 4 quarters
income_df['INCOME_GROWTH_MA4'] = income_df['INCOME_GROWTH'].rolling(4).mean()
#  Gross income growth last 4 quarters
income_mask = income_df['INCOME_GROWTH_MA4'] > 0.20

#  Calculate past 3 months
today = datetime.today()
today_date_str = today.strftime("%Y-%m-%d")
past_date = today - timedelta(days=90)
past_date_str = past_date.strftime("%Y-%m-%d")

#  Get last 3 months of reporting
income_mask &= ((income_df[REPORT_DATE] >= past_date_str) & (income_df[REPORT_DATE] <= today_date_str))

#  Format results
income_results = income_df[income_mask]
income_results = income_results.groupby(['Ticker']).mean()
income_results.reset_index(inplace=True)
income_results = income_results[['Ticker', 'INCOME_GROWTH_MA4']]
print(income_results)

In [ ]:
#  Get financial ratios
financial_ratios_df = hub.load_derived(variant='quarterly')  # annual or quarterly
financial_ratios_df = financial_ratios_df.reset_index()
for col in financial_ratios_df.columns:
    print(col)


#  Build the mask
#  Company’s current assets to its current liabilities.
financial_mask = (financial_ratios_df[CURRENT_RATIO] > 1.5)
# The debt ratio is defined as the ratio of total debt to total assets
financial_mask &= (financial_ratios_df[DEBT_RATIO] < 0.2)
financial_mask &= (financial_ratios_df[NET_PROFIT_MARGIN] > 0.15)
#  Get last 3 months of reporting
financial_mask &= ((financial_ratios_df[REPORT_DATE] >= past_date_str) & (financial_ratios_df[REPORT_DATE] <= today_date_str))

financial_selections_df = financial_ratios_df[financial_mask]
financial_results = financial_selections_df[['Ticker']]
print(financial_results)

In [ ]:
#  Price Ratio Screening
price_ratio_df = hub.load_derived_shareprices(variant='daily')  # daily or latest
price_ratio_df = price_ratio_df.reset_index()
for col in price_ratio_df.columns:
    print(col)

#  Build the mask
# The price-to-earnings ratio (P/E ratio) is the  current share price relative to its earnings per share (EPS)
price_ratio_mask = (price_ratio_df[PE_QUARTERLY] < 25)
#  The price-to-cash flow (P/CF) ratio compares a company's market value to its operating cash flow or
#  its stock price per share to operating cash flow per share
price_ratio_mask &= (price_ratio_df[PRICE_FCF_QUARTERLY] < 20)
#  Get last 3 months of reporting
price_ratio_mask &= ((price_ratio_df['Date'] >= past_date_str) & (price_ratio_df['Date'] <= today_date_str))

price_selections_df = price_ratio_df[price_ratio_mask]
price_results = price_selections_df.groupby(['Ticker']).mean()
price_results.reset_index(inplace=True)
price_results = financial_selections_df[['Ticker']]
print(price_results)

In [ ]:
from yahooquery import Ticker
import pandas as pd

def get_stock_info(ticker):
    """
    Get additional stock information using yahooquery.

    Parameters:
    - ticker: A yahooquery Ticker object.

    Returns:
    - ticker_info: A DataFrame containing stock information.
    """
    try:
        # Get summaryProfile and quoteType modules from yahooquery
        datasi = ticker.asset_profile
        print(datasi)

        # Convert the data to a DataFrame
        dfsi = pd.DataFrame.from_dict(datasi).T

        # Normalize the JSON data into a DataFrame for each module
        dataframes = [
            pd.json_normalize([x for x in dfsi[module] if isinstance(x, dict)])
            for module in ["summaryProfile", "quoteType"]
        ]

        # Concatenate the dataframes into a single dataframe
        dfsi = pd.concat(dataframes, axis=1)

        # Set the index of the dataframe to the 'symbol' column
        ticker_info = dfsi.set_index("symbol")

        return ticker_info
    except Exception as e:
        print(f"Error for {ticker} get_stock_info: {e}")
        pass



In [ ]:
from yahooquery import Ticker
ticker_obj = Ticker("AAPL")
ticker_obj.calendarEvents
